In [25]:
import pandas as pd

# Define the mapping of placeholder columns to actual columns for different comparison types
column_mappings = {
    'comparison_results_4o_Sum_to_4o_Raw.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_summary',
        'readability_file_2': 'readability_raw',
        'sentence_length_file_1': 'sentence_length_summary',
        'sentence_length_file_2': 'sentence_length_raw'
    },
    'comparison_results_35blog_sum_to_4oblog_sum.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_35t',
        'readability_file_2': 'readability_4o',
        'sentence_length_file_1': 'sentence_length_35t',
        'sentence_length_file_2': 'sentence_length_4o'
    },
    'comparison_results_raw_to_4o.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_raw',
        'readability_file_2': 'readability_blog',
        'sentence_length_file_1': 'sentence_length_raw',
        'sentence_length_file_2': 'sentence_length_blog'
    },
    'comparison_results_sum_to_3.5.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_summary',
        'readability_file_2': 'readability_blog',
        'sentence_length_file_1': 'sentence_length_summary',
        'sentence_length_file_2': 'sentence_length_blog'
    },
    'comparison_results_sum_to_4o_blog.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_summary',
        'readability_file_2': 'readability_blog',
        'sentence_length_file_1': 'sentence_length_summary',
        'sentence_length_file_2': 'sentence_length_blog'
    },
    'comparison_results_sum_to_raw.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_summary',
        'readability_file_2': 'readability_raw',
        'sentence_length_file_1': 'sentence_length_summary',
        'sentence_length_file_2': 'sentence_length_raw'
    },
    'comparison_results_raw_to_4o_sum_blog.csv': {
        'comparative_file_1': 'summary_file',
        'comparative_file_2': 'raw_file',
        'readability_file_1': 'readability_summary',
        'readability_file_2': 'readability_raw',
        'sentence_length_file_1': 'sentence_length_summary',
        'sentence_length_file_2': 'sentence_length_raw'
    }
}

# Function to rename columns based on the specific mapping for each file
def rename_columns(df, mapping):
    return df.rename(columns=mapping)

# List of files and their corresponding comparison types
files = [
    ('Individual Comparative CSVs/comparison_results_4o_Sum_to_4o_Raw.csv', '4o Sum Blo to 4o Raw Blog'),
    ('Individual Comparative CSVs/comparison_results_35blog_sum_to_4oblog_sum.csv', '35 Blog from Summary to 4o Blog from Summary'),
    ('Individual Comparative CSVs/comparison_results_raw_to_4o.csv', 'Raw Text to 4o Blog'),
    ('Individual Comparative CSVs/comparison_results_sum_to_3.5.csv', 'Summary to 35 Blog'),
    ('Individual Comparative CSVs/comparison_results_sum_to_4o_blog.csv', 'Summary to 4o Blog'),
    ('Individual Comparative CSVs/comparison_results_sum_to_raw.csv', 'Summary to Raw Text'),
    ('Individual Comparative CSVs/comparison_results_raw_to_4o_sum_blog.csv', 'Raw Text to 4o Blog from Summary')
]

# Load and standardize each dataframe
dataframes = []
for file, comparison_type in files:
    df = pd.read_csv(file)
    df = rename_columns(df, column_mappings[file.split('/')[-1]])
    df['comparison_type'] = comparison_type
    dataframes.append(df)

# Combine all dataframes into one unified dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

# Define metrics for calculation
metrics = [
    'exact_match_ratio', 'cosine_similarity', 'topic_overlap', 
    'key_phrase_overlap', 'semantic_similarity', 
    'readability_summary', 'readability_raw', 
    'sentence_length_summary', 'sentence_length_raw'
]

# Calculate differences for comparative columns (readability, sentence length)
combined_df['readability_difference'] = combined_df['readability_summary'] - combined_df['readability_raw']
combined_df['sentence_length_difference'] = combined_df['sentence_length_summary'] - combined_df['sentence_length_raw']

# Correct missing differences for specific comparison types
comparison_columns = {
    '35 Blog from Summary to 4o Blog from Summary': ['readability_35t', 'readability_4o', 'sentence_length_35t', 'sentence_length_4o'],
    'Raw Text to 4o Blog': ['readability_raw', 'readability_blog', 'sentence_length_raw', 'sentence_length_blog'],
    'Summary to 35 Blog': ['readability_summary', 'readability_blog', 'sentence_length_summary', 'sentence_length_blog'],
    'Summary to 4o Blog': ['readability_summary', 'readability_blog', 'sentence_length_summary', 'sentence_length_blog'],
    'Summary to 4o Blog': ['readability_summary', 'readability_blog', 'sentence_length_summary', 'sentence_length_blog']
}

for comparison_type, columns in comparison_columns.items():
    mask = (combined_df[columns[0]].notna()) & (combined_df[columns[1]].notna()) & (combined_df[columns[2]].notna()) & (combined_df[columns[3]].notna())
    combined_df.loc[mask, 'readability_difference'] = combined_df[columns[0]] - combined_df[columns[1]]
    combined_df.loc[mask, 'sentence_length_difference'] = combined_df[columns[2]] - combined_df[columns[3]]

# Save the updated dataframe to a new CSV file
combined_df.to_csv('combined_comparative_results.csv', index=False)

# Optional: Print summary of averages
print("Averages of metrics across all comparisons:")
print(combined_df[metrics].mean())


Averages of metrics across all comparisons:
exact_match_ratio           0.538195
cosine_similarity           0.813151
topic_overlap               0.583310
key_phrase_overlap          0.538195
semantic_similarity         0.945445
readability_summary        16.695200
readability_raw            11.573000
sentence_length_summary    28.596576
sentence_length_raw        36.462599
dtype: float64


## Calculate averages for each comparison type

In [26]:
import pandas as pd

# Load the combined dataframe
combined_df = pd.read_csv('combined_comparative_results.csv')

# Define metrics for calculation
metrics = [
    'exact_match_ratio', 'cosine_similarity', 'topic_overlap', 
    'key_phrase_overlap', 'semantic_similarity', 'readability_difference', 'sentence_length_difference'
]

# Calculate averages for each comparison type
average_metrics = combined_df.groupby('comparison_type')[metrics].mean().reset_index()

# Save the averages to a new CSV file
average_metrics.to_csv('average_metrics_by_comparison_type.csv', index=False)

# Display the average metrics
print(average_metrics)


                                comparison_type  exact_match_ratio  \
0  35 Blog from Summary to 4o Blog from Summary           0.634715   
1                     4o Sum Blo to 4o Raw Blog           0.424349   
2                           Raw Text to 4o Blog           0.156197   
3              Raw Text to 4o Blog from Summary           0.532964   
4                            Summary to 35 Blog           0.700610   
5                            Summary to 4o Blog           0.785568   
6                           Summary to Raw Text           0.532964   

   cosine_similarity  topic_overlap  key_phrase_overlap  semantic_similarity  \
0           0.881723       0.703333            0.634715             0.976759   
1           0.787325       0.577333            0.424349             0.966314   
2           0.772687       0.450000            0.156197             0.923666   
3           0.712451       0.439167            0.532964             0.890419   
4           0.922533       0.753333    